# Introduction

This Model is based on https://medium.com/nybles/create-your-first-image-recognition-classifier-using-cnn-keras-and-tensorflow-backend-6eaab98d14dd

Some changes applied as needed

# Installing Package 

If you are doing in google collaboration,
install list of packages:

!pip3 install tensorflow-gpu==1.13.1

!pip install numpy pandas

!pip install Keras

# Download Data Set

In [ ]:
!wget https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P14-Convolutional-Neural-Networks.zip

# Unzip Data Set

In [ ]:
!unzip P14-Convolutional-Neural-Networks.zip

## Building Model

In [ ]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
model = Sequential()

# Step 1 - Convolution
model.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
model.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
model.add(Convolution2D(32, 3, 3, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
model.add(Flatten())

# Step 4 - Full connection
model.add(Dense(output_dim = 128, activation = 'relu'))
model.add(Dense(output_dim = 1, activation = 'sigmoid'))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')
label_map = (training_set.class_indices)
label_map = dict((value,key) for key,value in label_map.items()) #flip key,value

#mapping label to JSON format and save to JSON file
import json

with open('label.json', 'w') as json_file:
  json.dump(label_map, json_file)

model.fit_generator(
    training_set,
    steps_per_epoch=4000,
    epochs=8,
    validation_data=test_set,
    validation_steps=400)

#uncomment this model if you want to try modelling with steps 8000 and epochs 10
"""
model.fit_generator(
    training_set,
    steps_per_epoch=8000,
    epochs=10,
    validation_data=test_set,
    validation_steps=800)
"""

In [0]:
# show the classification
print(label_map)

{0: 'cats', 1: 'dogs'}


# Export The Model

In [ ]:
from keras.models import model_from_json
import os

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# save model and architecture to single file
model.save("model_full.h5")
print("Saved model to disk")

'''
#if you want to know the type of the model and the data set given for training
print(type(model))
print(type(training_set))
print(type(test_set))
'''

# LOAD MODEL

In [ ]:
"""
# close your notebook or google collab and try to load your model directly before making prediction
# if you export your model separately into weight and the model itself, use this scripting
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
"""
import numpy
import os
from keras.models import load_model
model = load_model('model_full.h5')
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

# **Try Dog Image to be predicted**

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import json

#load file that will be predicted
def load_image(filename):
  # load the image
	img = load_img(filename, target_size=(64, 64))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 64, 64, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img

#load label from JSON file
label_place = r"C:\Users\Petra Febrianto L\Documents\Kaggle\Neurafarm\P14-Convolutional-Neural-Networks\Convolutional_Neural_Networks\Model_epoch4000\label.json"
with open(label_place, 'r') as json_file:
  label_image = json.load(json_file)

#make prediction
testing_image_place = r"C:\Users\Petra Febrianto L\Documents\Kaggle\Neurafarm\P14-Convolutional-Neural-Networks\Convolutional_Neural_Networks\dog.jpg"
testing_image = load_image(testing_image_place)
result_binary = model.predict(testing_image) #result in binary, our classification method is binary


#training_set.class_indices
#convert binary to name class
if result_binary[0][0] >= 0.5:
    prediction = label_image["1"]
else:
    prediction = label_image["0"]

print(prediction)